In [23]:
import typing as tp

import numpy as np

In [35]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None) -> None:
        self.feature: tp.Optional[int] = feature
        self.threshold: tp.Optional[float] = threshold
        self.left: tp.Optional[Node] = left
        self.right: tp.Optional[Node] = right
        self.value: tp.Optional[np.signedinteger] = value

    def is_leaf(self) -> bool:
        return self.value is not None


class CustomDecisionTree:
    def __init__(self, max_depth: tp.Optional[int] = None, min_samples_split: int = 2,
                 min_samples_leaf: int = 1) -> None:
        self.max_depth: tp.Optional[int] = max_depth
        self.min_samples_to_split: int = min_samples_split
        self.min_samples_leaf: int = min_samples_leaf
        self.tree: tp.Optional[Node] = None
        self.cur_depth: tp.Optional[int] = 0

    def fit(self, X: np.ndarray, y: np.ndarray) -> None:
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X: np.ndarray, y: np.ndarray, depth: int = 0) -> Node:
        n_samples, n_features = X.shape
        self.n_classes = len(np.unique(y))
        if self.max_depth and depth >= self.max_depth \
                or self.n_classes == 1 \
                or n_samples < self.min_samples_to_split:
            leaf_value: np.signedinteger = self._most_common_label(y)
            return Node(value=leaf_value)

        rand_features = np.random.choice(n_features, n_features, replace=False)
        best_feature, best_threshold = self._best_criteria(X, y, rand_features)
        left_idxs, right_idxs = self._split(X[:, best_feature], best_threshold)

        if len(left_idxs) < self.min_samples_leaf or len(right_idxs) < self.min_samples_leaf:
            leaf_value: np.signedinteger = self._most_common_label(y)
            return Node(value=leaf_value)
        self.cur_depth += 1
        left = self._build_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._build_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(feature=best_feature, threshold=best_threshold, left=left, right=right)

    def _best_criteria(self, X: np.ndarray, y: np.ndarray, features: np.ndarray) -> tuple[int, float]:
        best_gain = -1
        best_feature, best_threshold = None, None
        for feature in features:
            X_column_by_feature = X[:, feature]
            thresholds = np.unique(X_column_by_feature)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column_by_feature=X_column_by_feature, split_thresh=threshold)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        return best_feature, best_threshold

    def _information_gain(self, y, X_column_by_feature: np.ndarray, split_thresh: float) -> float:
        parent_entropy = self._calc_entropy(y)
        left_idxs, right_idxs = self._split(X_column_by_feature, split_thresh)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        n_total, n_left, n_right = len(y), len(left_idxs), len(right_idxs)
        entropy_left, entropy_right = self._calc_entropy(y[left_idxs]), self._calc_entropy(y[right_idxs])
        k_left, k_right = n_left / n_total, n_right / n_total
        child_entropy = k_left * entropy_left + k_right * entropy_right
        ig = parent_entropy - child_entropy
        return ig

    @staticmethod
    def _split(X_column_by_feature: np.ndarray, split_thresh: float) -> tuple[np.ndarray, np.ndarray]:
        left_idxs = np.argwhere(X_column_by_feature <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column_by_feature > split_thresh).flatten()
        return left_idxs, right_idxs

    @staticmethod
    def _calc_entropy(y: np.ndarray) -> float:
        p_lst = np.bincount(y) / len(y)
        return -np.sum([p * np.log2(p) for p in p_lst if p > 0])

    @staticmethod
    def _most_common_label(y: np.ndarray) -> np.signedinteger:
        return np.bincount(y).argmax()

    def predict(self, X: np.ndarray) -> np.ndarray:
        return np.array([self._traverse_tree(x, self.tree) for x in X])

    def _traverse_tree(self, x, node) -> np.signedinteger:
        if node.is_leaf():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)


In [36]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer


In [37]:
def _get_split_data() -> tp.Any:
    data = load_breast_cancer()
    X, y = data.data, data.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_test, X_train, y_test, y_train

In [38]:
def pretty_print_best_params(best_params: dict, best_score: float) -> None:
    print(f"Best Score: {best_score:.4f}")
    print("Best Parameters:")
    for key, value in best_params.items():
        print(f"  {key}: {value}")


def print_accuracy(y_pred, y_test) -> None:
    test_accuracy = accuracy_score(y_test, y_pred)
    print(f"Test Accuracy: {test_accuracy}")

In [39]:
def test_decision_tree() -> None:
    X_test, X_train, y_test, y_train = _get_split_data()
    dt = CustomDecisionTree()
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)
    print_accuracy(y_pred, y_test)

In [40]:
test_decision_tree()

In [41]:
import pandas as pd

In [42]:
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df

In [43]:
data = load_breast_cancer()
X = data.data
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [47]:
import matplotlib.pyplot as plt

In [49]:
def plot_tree_depth_by_params(
        min_samples_split_lst,
        min_samples_leaf_lst,
        X_train,
        y_train
):
    depth_lst = []
    labels = []

    for min_samples_split in min_samples_split_lst:
        for min_samples_leaf in min_samples_leaf_lst:
            dt = CustomDecisionTree(min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
            dt.fit(X_train, y_train)
            max_depth = dt.cur_depth
            depth_lst.append(max_depth)
            labels.append(
                f"split:{min_samples_split};leaf:{min_samples_leaf}"
            )
    plt.figure(figsize=(10, 6))
    plt.plot(depth_lst)
    plt.xticks(range(len(depth_lst)), labels, rotation=45, ha='right')
    plt.xlabel("Hyperparameters")
    plt.ylabel("Tree depth")
    plt.title("Tree depth by hyperparameters")
    plt.show()

In [51]:
# min_samples_split_lst = [i for i in range(2, 31)]
min_samples_split_lst = [2, 3, 5, 10, 15, 25]
# min_samples_leaf_lst = [i for i in range(1, 31)]
min_samples_leaf_lst = [2, 3, 5, 10, 15, 25]

plot_tree_depth_by_params(
    min_samples_split_lst=min_samples_split_lst,
    min_samples_leaf_lst=min_samples_leaf_lst,
    X_train=X_train,
    y_train=y_train
)


In [ ]:
import optuna
import numpy as np


In [32]:
def func(trial, X_train: np.ndarray, X_val: np.ndarray, y_train: np.ndarray, y_val: np.ndarray) -> float:
    max_depth = trial.suggest_int('max_depth', 2, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 30)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 30)
    dt = CustomDecisionTree(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
    )
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

In [33]:
def get_best_hyperparameters(
        X_train: np.ndarray, X_val: np.ndarray, y_train: np.ndarray, y_val: np.ndarray
) -> tuple[dict, float]:
    study = optuna.create_study(direction='maximize')
    study.optimize(
        func=lambda trial: func(trial, X_train, X_val, y_train, y_val),
        n_trials=100,
        show_progress_bar=True
    )
    best_params = study.best_params
    best_value = study.best_value
    return best_params, best_value


In [34]:
def get_split_data():
    data = load_breast_cancer()
    X, y = data.data, data.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    return X_test, X_train, X_val, y_test, y_train, y_val

In [35]:
X_test, X_train, X_val, y_test, y_train, y_val = get_split_data()
best_params, best_score = get_best_hyperparameters(X_train, X_val, y_train, y_val)
print("~" * 40)
pretty_print_best_params(best_params, best_score)

In [36]:
import matplotlib.pyplot as plt

In [37]:
def plot_accuracy_vs_depth(X_test, X_train, y_test, y_train,
                           min_samples_split, min_samples_leaf, depth_lst):
    train_accuracies = []
    test_accuracies = []

    for depth in depth_lst:
        dt = CustomDecisionTree(
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_depth=depth,
        )
        dt.fit(X_train, y_train)

        y_train_pred = dt.predict(X_train)
        train_accuracies.append(accuracy_score(y_train, y_train_pred))

        y_test_pred = dt.predict(X_test)
        test_accuracies.append(accuracy_score(y_test, y_test_pred))

    plt.plot(depth_lst, train_accuracies, label='Train Accuracy')
    plt.plot(depth_lst, test_accuracies, label='Test Accuracy')
    plt.xlabel('Depth')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs Depth')
    plt.legend()
    plt.show()

In [38]:
depth_lst = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 25]
plot_accuracy_vs_depth(
    depth_lst=depth_lst,
    X_test=X_test,
    X_train=X_train,
    y_test=y_test,
    y_train=y_train,
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf']
)